In [1]:
import pandas as pd
import os

In [3]:
import langchain
import pandas as pd

In [4]:
folder_name = "MasterData"
file_path_4 = os.path.join(folder_name, f"Merged_MasterData.xlsx")
metadata_df = pd.read_excel(file_path_4)
metadata_df[:3]

,Report_Name,Page_Name,Visual_Title,Visual_Type,Columns,Responses
0,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,funnel,"Sales Stage, Revenue Open",The Revenue Open by Sales Stage visual in Powe...
1,POC_Regional_Sales,Sales Overview,Forecast by Location,shapeMap,"State or Province, State or Province.1, Rev Go...",The Power BI visual Forecast by Location is a ...
2,POC_Regional_Sales,Sales Overview,Revenue Won and Revenue In Pipeline by Product...,barChart,"Product Category, Product, Revenue Won, Revenu...","The Power BI visual ""Revenue Won and Revenue I..."


In [7]:
#from langchain.retrievers.multi_vector import MultiVectorRetriever

In [ ]:
# Initialize the MultiVector Retriever
retriever = langchain.MultiVectorRetriever(
    vectorstore=langchain.Chroma(
        collection_name="descriptions",
        embedding_function=langchain.OpenAIEmbeddings(),
    ),
    docstore=langchain.InMemoryStore(),
    id_key="Report_Name",
)

# Add the descriptions to the vectorstore
for i, row in metadata_df.iterrows():
    description = row["Description"]
    report_name = row["Report_Name"]
    retriever.vectorstore.add_documents([langchain.Document(page_content=description, metadata={"Report_Name": report_name})])

def retrieve_similar_descriptions(query):
    """Retrieves the top 3 descriptions that are similar to the given query."""
    return retriever.search(query, k=3)

# Example usage
query = "What are the sales figures for the North region?"
similar_descriptions = retrieve_similar_descriptions(query)

for description in similar_descriptions:
    print(f"Report Name: {description.metadata['Report_Name']}")
    print(f"Visual Title: {description.metadata['Visual_Title']}")
    print(f"Description: {description.page_content}")
    print("---------------------------------")


In [10]:
#pip install sentence_transformers

In [11]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

In [12]:
folder_name = "MasterData"
file_path_4 = os.path.join(folder_name, f"Merged_MasterData.xlsx")
metadata_df = pd.read_excel(file_path_4)
metadata_df[:3]

,Report_Name,Page_Name,Visual_Title,Visual_Type,Columns,Responses
0,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,funnel,"Sales Stage, Revenue Open",The Revenue Open by Sales Stage visual in Powe...
1,POC_Regional_Sales,Sales Overview,Forecast by Location,shapeMap,"State or Province, State or Province.1, Rev Go...",The Power BI visual Forecast by Location is a ...
2,POC_Regional_Sales,Sales Overview,Revenue Won and Revenue In Pipeline by Product...,barChart,"Product Category, Product, Revenue Won, Revenu...","The Power BI visual ""Revenue Won and Revenue I..."


In [13]:
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
description_embeddings = model.encode(metadata_df["Responses"].astype(str), convert_to_tensor=True)

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
deprecated() got an unexpected keyword argument 'name'

In [ ]:




# Build an index using faiss
index = faiss.IndexFlatIP(description_embeddings.size(1))
index.add(description_embeddings.numpy())

# Function to perform similarity search
def get_top_matches(query, top_k=3):
    query_embedding = model.encode(query, convert_to_tensor=True).numpy()
    _, similar_indices = index.search(query_embedding, top_k)
    return metadata_df.iloc[similar_indices[0]]

# Example usage
user_query = "Your user query here"
top_matches = get_top_matches(user_query)
print(top_matches[["Report_Name", "Page_Name", "Visual_Title", "Visual_Type", "Columns", "Description"]])
